<a href="https://colab.research.google.com/github/FaridRash/IGNIS/blob/main/snr_calculator_modtran_final_scaled.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
https://wp.optics.arizona.edu/alumni/wp-content/uploads/sites/113/2024/06/Jacob_Wilson_Master_s_Report_2024.pdf
maybe when you do the calculation cite that the value is taken from this source
"""

'\nhttps://wp.optics.arizona.edu/alumni/wp-content/uploads/sites/113/2024/06/Jacob_Wilson_Master_s_Report_2024.pdf\nmaybe when you do the calculation cite that the value is taken from\xa0this\xa0source\n'

In [2]:

"""
For the campi flegrei target we are considering a 200 km^2 square area (approximately 14x14km).
But the specific spots we want to investigate are the fumarole (whose vents can reach 10 metres and 165degree)
Consider those would be much smaller than our gsd, so if you consider the signal from that area it would be a
weighted sum of the signal from the fumarola (~10m, 150-160 °C) and from the sourrunding ground (GSD-10m, Ground Temperature)

But this thing of the weighted sum can be implemented later,
for now let’s see what happens with the whole campi flegrei as target,
then let’s consider the hotspot, then let’s consider the weighted sum (if necessary)
"""

'\nFor the campi flegrei target we are considering a 200 km^2 square area (approximately 14x14km).\nBut the specific spots we want to investigate are the fumarole (whose vents can reach 10 metres and 165degree)\nConsider those would be much smaller than our gsd, so if you consider the signal from that area it would be a\nweighted sum of the signal from the fumarola (~10m, 150-160 °C) and from the sourrunding ground (GSD-10m, Ground\xa0Temperature)\n\nBut this thing of the weighted sum can be implemented later,\nfor now let’s see what happens with the whole campi flegrei as target,\nthen let’s consider the hotspot, then let’s consider the weighted sum\xa0(if\xa0necessary)\n'

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

In [4]:
c1 = 1.191042972e-16
c2 = c2 = 1.4388e-2
h = 6.626e-34
c = 3e8

In [5]:
def compute_snr(T, theta_deg, R, tau_a):
    lambda_m = lambda_range * 1e-6
    cos_theta = np.cos(np.deg2rad(theta_deg))
    exponent = np.clip(c2 / (lambda_m * T), 0, 700)  # cap large exponents
    #print(f'exponent {exponent}')
    M_lambda = c1 / (lambda_m**5 * (np.exp(exponent) - 1))
    #print(f'M_lambda: {M_lambda}')
    E_lambda = epsilon * M_lambda
    #print(f'E_lambda: {E_lambda}')
    S_lambda = lambda_m * E_lambda * tau_a * tau_0 * eta * delta_lambda
    #print(S_lambda)
    S_sum = np.sum(S_lambda)
    #print(S_sum)
    S_total = (D**2 * t * cos_theta * A0) / (4 * R**2 * h * c) * S_sum
    #print(S_total)

    N_total = np.sqrt(S_total + NEDT)
    SNR_total = S_total / N_total
    SNR_lambda = S_lambda / N_total

    plt.figure(figsize=(10, 5))
    plt.plot(lambda_m, SNR_lambda, label='SNR Contribution per λ', color='blue')
    plt.xlabel('Wavelength (μm)')
    plt.ylabel('SNR Contribution')
    plt.title(f'Spectral SNR Contribution\nT = {T}K, θ = {theta_deg}°, R = {R/1000:.0f}km, τₐ = {tau_a}, A0 = {A0}m' )
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

    print(f"Total SNR at T = {T} K, θ = {theta_deg}°, R = {R/1000:.0f} km, τₐ = {tau_a}: {SNR_total:.2f}")

In [ ]:
epsilon = 0.95
tau_0 = 0.89
eta = 0.7
t = 0.01
A0 = 100
focal_length = 72.8
F_number = 1.05
D = (focal_length / F_number) / 1000
NEDT = 0.0441

lambda_range = np.arange(8, 14.1, 0.1)
delta_lambda = 0.1e-6  # meters


In [ ]:
interact(
    compute_snr,
    T=FloatSlider(value=438.0, min=200, max=500, step=1, description='Temperature (K)'),
    theta_deg=FloatSlider(value=0, min=0, max=90, step=1, description='Theta (°)'),
    R=FloatSlider(value=460000, min=150000, max=460000, step=1000, description='Distance (m)'),
    tau_a=FloatSlider(value=0.85, min=0.0, max=1.0, step=0.01, description='τₐ (Atmosphere)')

)